# 02 Rule Diagnostics: EMAクロスの失敗パターンを理解する

このノートブックでは、**EMAクロス戦略がなぜダマし（whipsaw）に弱いのか**、そして**ATRベースのレジームフィルタがどう役立つか**を、数値と図で確認します。

## What you will learn
- EMAクロスが発生する仕組みと、遅れ（lag）が生じる理由
- ATRを使った「アクティブ相場 / 非アクティブ相場」の判定方法
- シグナル頻度、アクティブ比率、whipsaw回数の診断方法
- 大きな値動きの前後で、EMAがどのように反応するかの観察ポイント


## 2) 用語集（Glossary）

- **OHLC**: Open, High, Low, Close（始値・高値・安値・終値）
- **return（リターン）**: 価格変化率（通常は日次収益率）
- **EMA（指数移動平均）**: 直近データに大きな重みを置く移動平均
- **ATR（Average True Range）**: 値動きの大きさ（ボラティリティ）を表す指標
- **regime（レジーム）**: 相場の状態（例: 動きが大きい/小さい）
- **whipsaw（ダマし）**: 売買シグナルが短期間で反転し続ける現象
- **lag（遅れ）**: 指標が価格変化に追いつくまでの時間差


## 3) データ読み込み

ここでは以下のノブ（調整パラメータ）を用意します。
- `SYMBOL`: 取得する銘柄（デフォルトは `QQQ`）
- `PERIOD`, `INTERVAL`: 取得期間と足種

`quantlab.data.fetch_ohlc` でOHLCデータを取得し、
- 日付範囲
- 行数
- 欠損値（NaN）
を sanity check として表示します。


In [ ]:
# --- 3) データ読み込み ---
# プロジェクトのsrcをimport可能にする（ノートブック単体実行のため）
from pathlib import Path
import sys

PROJECT_ROOT = Path.cwd().resolve()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent
SRC_DIR = PROJECT_ROOT / "src"
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from quantlab.data import fetch_ohlc
from quantlab.indicators import ema, atr

# ノブ（まずはデフォルト値で実行し、後で変更して比較してみましょう）
SYMBOL = "QQQ"
PERIOD = "3y"
INTERVAL = "1d"

# データ取得
# ネットワーク制約でyfinanceが失敗する環境向けに、教育用フォールバックを用意します。
try:
    df = fetch_ohlc(SYMBOL, period=PERIOD, interval=INTERVAL).copy()
    print("Data source: quantlab.data.fetch_ohlc (yfinance)")
except Exception as e:
    print(f"[WARN] fetch_ohlc failed: {e}")
    print("[WARN] Falling back to synthetic OHLC data so the notebook can run end-to-end.")

    # 3年分の営業日を作成し、ランダムウォークからCloseを生成
    rng = np.random.default_rng(42)
    idx = pd.bdate_range(end=pd.Timestamp.today().normalize(), periods=756)
    daily_ret = rng.normal(loc=0.0004, scale=0.012, size=len(idx))
    close = 100 * (1 + pd.Series(daily_ret, index=idx)).cumprod()

    # OHLCの整合性を保つため、Closeを中心にOpen/High/Lowを合成
    open_ = close.shift(1).fillna(close.iloc[0]) * (1 + rng.normal(0, 0.002, len(idx)))
    spread = close * rng.uniform(0.002, 0.02, len(idx))
    high = pd.concat([open_, close], axis=1).max(axis=1) + spread / 2
    low = pd.concat([open_, close], axis=1).min(axis=1) - spread / 2
    volume = pd.Series(rng.integers(1_000_000, 8_000_000, len(idx)), index=idx)

    df = pd.DataFrame({
        "Open": open_.values,
        "High": high.values,
        "Low": low.values,
        "Close": close.values,
        "Volume": volume.values,
    }, index=idx)

# 日付インデックスを扱いやすくする（yfinanceの仕様差を吸収）
df.index = pd.to_datetime(df.index)

print(f"SYMBOL: {SYMBOL}")
print(f"Rows: {len(df):,}")
print(f"Date range: {df.index.min().date()} -> {df.index.max().date()}")
print("\nNaN counts:")
print(df.isna().sum())

# 最後の10行を小さな表として確認
print("\nTail (OHLC key columns):")
print(df[["Open", "High", "Low", "Close"]].tail(10))

## 4) 指標計算

ここでは以下を計算します。
- `EMA_FAST`, `EMA_SLOW`
- `ATR_PERIOD`, `REGIME_WIN`
- `ema_diff = EMA_fast - EMA_slow`
- `atr_thresh = ATRのローリング中央値（窓: REGIME_WIN）`
- `active = ATR > atr_thresh`

`active=False` のときは、シグナルの信頼性が低い（横ばいでダマしが増えやすい）と仮定します。


In [ ]:
# --- 4) 指標計算 ---
# ノブ：EMAとATRのパラメータ
EMA_FAST = 12
EMA_SLOW = 26
ATR_PERIOD = 14
REGIME_WIN = 60

work = df.copy()

# EMA（短期・長期）
work["EMA_fast"] = ema(work["Close"], EMA_FAST)
work["EMA_slow"] = ema(work["Close"], EMA_SLOW)

# ATRとその閾値（過去REGIME_WINの中央値）
work["ATR"] = atr(work, ATR_PERIOD)
work["atr_thresh"] = work["ATR"].rolling(REGIME_WIN).median()

# EMA差分（クロス判定の中心）
work["ema_diff"] = work["EMA_fast"] - work["EMA_slow"]

# レジーム判定：ATRが閾値を超えていればactive
work["active"] = work["ATR"] > work["atr_thresh"]

# 日次リターン（lag観察で使用）
work["ret"] = work["Close"].pct_change()

print("Computed columns preview:")
print(work[["Close", "EMA_fast", "EMA_slow", "ema_diff", "ATR", "atr_thresh", "active"]].tail(10))

## 5) 数式で理解する（display math only）

EMAの再帰式：

$$
\mathrm{EMA}_t = lpha \cdot P_t + (1-lpha) \cdot \mathrm{EMA}_{t-1},\quad lpha = rac{2}{n+1}
$$

True Range（TR）とATR：

$$
\mathrm{TR}_t = \max\left( H_t-L_t,\ |H_t-C_{t-1}|,\ |L_t-C_{t-1}| ight)
$$

$$
\mathrm{ATR}_t = rac{1}{n}\sum_{i=0}^{n-1} \mathrm{TR}_{t-i}
$$

クロスオーバー条件（`ema_diff` の符号変化）：

$$
\mathrm{ema\_diff}_t = \mathrm{EMA}^{\mathrm{fast}}_t - \mathrm{EMA}^{\mathrm{slow}}_t
$$

$$
	ext{BUY at }t\iff \mathrm{ema\_diff}_{t-1}\le 0\ \land\ \mathrm{ema\_diff}_t>0
$$

$$
	ext{SELL at }t\iff \mathrm{ema\_diff}_{t-1}\ge 0\ \land\ \mathrm{ema\_diff}_t<0
$$


## 6) Diagnostics

### A) Signal frequency（BUY/SELL/HOLD件数）
### B) Active ratio（activeの割合）
### C) Whipsaw detector（N日以内に反対シグナルへ反転）
### D) Lag intuition（大きな値動き前後でEMAの反応を確認）


In [ ]:
# --- 6A/B/C) シグナル診断 ---
# シグナルを作るための前日値
prev_diff = work["ema_diff"].shift(1)

# クロス条件（生のクロス）
cross_up = (prev_diff <= 0) & (work["ema_diff"] > 0)
cross_down = (prev_diff >= 0) & (work["ema_diff"] < 0)

work["raw_signal"] = np.select(
    [cross_up, cross_down],
    ["BUY", "SELL"],
    default="HOLD",
)

# フィルタ後シグナル：activeでない日はHOLDへ抑制
work["signal"] = np.where(work["active"], work["raw_signal"], "HOLD")

# A) シグナル頻度
freq = work["signal"].value_counts().reindex(["BUY", "SELL", "HOLD"], fill_value=0)
print("A) Signal frequency (filtered):")
print(freq.to_frame("count"))

# 参考：raw_signalも比較表示
raw_freq = work["raw_signal"].value_counts().reindex(["BUY", "SELL", "HOLD"], fill_value=0)
print("\nReference - raw signal frequency (no regime filter):")
print(raw_freq.to_frame("count"))

# B) active比率
active_ratio = work["active"].mean() * 100
print(f"\nB) Active ratio: {active_ratio:.2f}%")

# C) whipsaw検出
# 定義：最後の非HOLDシグナルからN日以内に逆方向シグナルが出たらwhipsaw候補と数える
WHIPSAW_N = 5  # ノブ：短期反転を何日以内とみなすか

events = work.loc[work["signal"].isin(["BUY", "SELL"]), ["signal"]].copy()

whipsaw_count = 0
whipsaw_rows = []
for i in range(1, len(events)):
    prev_idx = events.index[i - 1]
    curr_idx = events.index[i]
    prev_sig = events.iloc[i - 1]["signal"]
    curr_sig = events.iloc[i]["signal"]

    # 営業日ベースの行距離を使って近接度を判定
    bars_apart = work.index.get_loc(curr_idx) - work.index.get_loc(prev_idx)
    is_alternating = prev_sig != curr_sig

    if is_alternating and bars_apart <= WHIPSAW_N:
        whipsaw_count += 1
        whipsaw_rows.append((prev_idx.date(), prev_sig, curr_idx.date(), curr_sig, bars_apart))

print(f"\nC) Whipsaw count within {WHIPSAW_N} bars: {whipsaw_count}")
if whipsaw_rows:
    whipsaw_df = pd.DataFrame(
        whipsaw_rows,
        columns=["prev_date", "prev_signal", "curr_date", "curr_signal", "bars_apart"],
    )
    print(whipsaw_df.head(10))
else:
    print("No whipsaw events found under current parameters.")


In [ ]:
# --- 6D) Lag intuition: 大きな値動き周辺を観察 ---
TOP_K = 3     # 何件の大きな値動きを見るか
WINDOW = 5    # 前後何日を表示するか

# 絶対リターン上位日を抽出（NaN除外）
shock_days = work["ret"].abs().dropna().nlargest(TOP_K)

print("D) Largest |daily return| days and local windows:")
for dt, val in shock_days.items():
    print(f"\n--- Shock day: {dt.date()} |return|={val:.2%} ---")
    loc = work.index.get_loc(dt)
    start = max(0, loc - WINDOW)
    end = min(len(work), loc + WINDOW + 1)

    cols = ["Close", "ret", "EMA_fast", "EMA_slow", "ema_diff", "signal", "active"]
    snippet = work.iloc[start:end][cols]
    print(snippet)


## 7) Plots

- 価格 + EMA（クロスポイントをマーク）
- ATR + 閾値 + active領域のシェーディング


In [ ]:
# --- 7) 可視化 ---
plot_df = work.dropna(subset=["EMA_fast", "EMA_slow", "ATR", "atr_thresh"]).copy()

# クロスポイント抽出
plot_prev = plot_df["ema_diff"].shift(1)
plot_buy = (plot_prev <= 0) & (plot_df["ema_diff"] > 0)
plot_sell = (plot_prev >= 0) & (plot_df["ema_diff"] < 0)

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# 上段：価格とEMA
axes[0].plot(plot_df.index, plot_df["Close"], label=f"{SYMBOL} Close", color="black", linewidth=1.3)
axes[0].plot(plot_df.index, plot_df["EMA_fast"], label=f"EMA{EMA_FAST}", color="tab:blue", alpha=0.9)
axes[0].plot(plot_df.index, plot_df["EMA_slow"], label=f"EMA{EMA_SLOW}", color="tab:orange", alpha=0.9)

axes[0].scatter(plot_df.index[plot_buy], plot_df.loc[plot_buy, "Close"], marker="^", color="green", s=45, label="Cross Up")
axes[0].scatter(plot_df.index[plot_sell], plot_df.loc[plot_sell, "Close"], marker="v", color="red", s=45, label="Cross Down")

axes[0].set_title(f"{SYMBOL}: Price + EMA Crossover Points")
axes[0].set_ylabel("Price")
axes[0].legend(loc="upper left")
axes[0].grid(alpha=0.25)

# 下段：ATRと閾値 + activeのシェーディング
axes[1].plot(plot_df.index, plot_df["ATR"], label=f"ATR({ATR_PERIOD})", color="tab:purple")
axes[1].plot(plot_df.index, plot_df["atr_thresh"], label=f"Median ATR ({REGIME_WIN})", color="tab:gray", linestyle="--")

# active=True の区間を薄く塗る
axes[1].fill_between(
    plot_df.index,
    0,
    plot_df["ATR"].max() * 1.05,
    where=plot_df["active"].astype(bool),
    color="gold",
    alpha=0.15,
    label="Active regime",
)

axes[1].set_title("ATR Regime Filter")
axes[1].set_ylabel("ATR")
axes[1].set_xlabel("Date")
axes[1].legend(loc="upper left")
axes[1].grid(alpha=0.25)

plt.tight_layout()
plt.show()

## 8) What to observe（観察チェックリスト）

- EMAクロスは**トレンド局面では有効**だが、横ばいでは反転が増えやすいか？
- `active=False` の期間にクロスが多発していないか？
- フィルタ適用で `BUY/SELL` 件数はどれくらい減るか？
- `WHIPSAW_N` を変えると、whipsaw件数はどう変化するか？
- 大きなリターン日の前後で、EMAが追随するまでの遅れ（lag）が見えるか？


## 9) Mini exercises（先に予想してから実行）

1. `EMA_FAST=8`, `EMA_SLOW=34` に変更すると、シグナル頻度とwhipsaw件数はどう変わる？
2. `ATR_PERIOD=7` に短縮すると、`active` 判定は敏感になる？鈍感になる？
3. `REGIME_WIN=120` にすると、閾値は安定する？それとも反応が遅くなる？
4. `WHIPSAW_N=3` と `WHIPSAW_N=10` を比較し、戦略の「騒がしさ」をどう評価する？
5. `SYMBOL` を `SPY`, `IWM` に変えて、同じ設定で挙動差を説明してみよう。

> 学習のコツ: **変更前に「増える/減る」を言語化**し、結果と照合すると理解が深まります。
